# Data Assimilation with Newtonian Nudging 

In [ ]:
import os
from pathlib import Path
pad = Path(os.getcwd())
if pad.name == 'data_assimilation':
    pad_correct = pad.parent
    os.chdir(pad_correct)
from functions.PDM import PDM
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import hvplot 
import hvplot.pandas
from numba import jit

%load_ext autoreload 
%autoreload 2 

In [ ]:
#Needed for PDM inputs
preprocess_output_folder = Path('data/Zwalm_data/preprocess_output')
p_zwalm = pd.read_pickle(preprocess_output_folder / 'zwalm_p_thiessen.pkl')
ep_zwalm = pd.read_pickle(preprocess_output_folder / 'zwalm_ep_thiessen.pkl')
param = pd.read_csv("data/Zwalm_PDM_parameters/NM_opt_param.csv")
zwalm_shape = gpd.read_file('data/Zwalm_shape/zwalm_shapefile_emma_31370.shp')
area_zwalm_new = np.single(zwalm_shape.area[0] / 10**6)
deltat = np.array(1, dtype=np.float32)  # hour
deltat_out = np.array(24, dtype=np.float32)  # daily averaging
deltat = np.array(1, dtype=np.float32)  # hour
deltat_out = np.array(24, dtype=np.float32)  # daily averaging

#observational C*
ml_obs_op_pad = Path("data/ml_obs_op_data")
C_star_obs_lin_reg = pd.read_pickle(ml_obs_op_pad/'lin_reg/full_data/y_hat_retimed.pickle')

#Observational flow for comparison
Q_obs_daily = pd.read_pickle('data/Zwalm_data/pywaterinfo_output/Q_day.pkl')
Q_obs_daily = Q_obs_daily.rename(columns = {'Timestamp':'t'})
Q_obs_daily = Q_obs_daily.set_index('t')
Q_obs_daily.head()

In [ ]:
pd_zwalm_out_hour = PDM(P=p_zwalm['P_thiessen'].values,
                       EP=ep_zwalm['EP_thiessen'].values,
                       t=p_zwalm['Timestamp'].values,
                       area=area_zwalm_new, deltat=deltat, deltatout=deltat_out,
                       parameters=param, m=3, DA = True, Cstar_obs = C_star_obs_lin_reg.values.flatten(),t_obs = C_star_obs_lin_reg.index.values, gamma = 0.5, kappa = 1,  tau = np.timedelta64(12,'h'),)
pd_zwalm_out_hour_DA = pd_zwalm_out_hour.set_index('Time')
pd_zwalm_out_hour = PDM(P=p_zwalm['P_thiessen'].values,
                       EP=ep_zwalm['EP_thiessen'].values,
                       t=p_zwalm['Timestamp'].values,
                       area=area_zwalm_new, deltat=deltat, deltatout=deltat_out,
                       parameters=param, m=3, DA = False)
pd_zwalm_out_hour = pd_zwalm_out_hour.set_index('Time')

In [ ]:
fig, ax = plt.subplots()
pd_zwalm_out_hour_DA['Cstar'].plot(ylabel='[mm]', ax = ax, label = 'C* DA')
pd_zwalm_out_hour['Cstar'].plot(ax = ax, label = 'C* OL')
ax.legend()
pd_zwalm_out_hour.tail()

In [ ]:
pd_zwalm_out_hour_DA['Cstar'].hvplot(ylabel='[mm]',
    label = 'C* DA')*pd_zwalm_out_hour['Cstar'].hvplot(label = 'C* OL')

In [ ]:
Q_obs_daily['Value'].hvplot()

In [ ]:
Q_obs_daily['Value'].hvplot(label = 'Observed')*pd_zwalm_out_hour_DA[
    'qmodm3s'].hvplot(ylabel='[m^3/s]',label = 'DA')*pd_zwalm_out_hour['qmodm3s'].hvplot(label = 'OL',line_dash = 'dotted', frame_width = 800, frame_height = 400)

In [ ]:
Q_out_diff = pd_zwalm_out_hour_DA['qmodm3s'] - pd_zwalm_out_hour['qmodm3s']
Q_out_diff.hvplot()

# Old experimens only below 

In [ ]:
t_hour = p_zwalm['Timestamp'].values.astype('datetime64[h]')
t_obs = C_star_obs_lin_reg.index.values.astype('datetime64[h]') 
C_star_mod = pd_zwalm_out_hour['Cstar']

In [ ]:
def tau_weighing(delta_t_abs, tau):
    if delta_t_abs < tau/2:
        W_t = 1
    elif delta_t_abs < tau:
        W_t = (tau - delta_t_abs)/(tau/2)
    else:
        W_t = 0
    return W_t
weights = [tau_weighing(np.abs(delta_t), 10) for delta_t in np.arange(-20,20,1)]
plt.plot(weights)

In [ ]:
np.arange(-30,30)

In [ ]:
def NewtonianNudging(Cstar_min, Cstar_obs, gamma, Kappa, delta_t, tau):
    W_t = tau_weighing(np.abs(delta_t),tau)
    Cstar_plus = Cstar_min + gamma*Kappa*W_t*(Cstar_obs -Cstar_min)
    if Cstar_plus != Cstar_min:
        import pdb; pdb.set_trace()
    return Cstar_plus


In [ ]:
def NN_wrapper(i, t, t_obs, t_a,Cstar,C_star_obs):
    if np.any(np.abs(t[i] - t_obs) < t_a):
        t_assimilated=t_obs[np.abs((t[i] - t_obs)) < t_a]
        print(
            f'{t[i]} should be assimilated since less than {t_a/2} rmeoved from {t_assimilated}')

In [ ]:
C_star_updated = C_star_mod.copy()
for i in range(len(C_star_mod)):
    C_star_min = C_star_mod[i]
    t_mod_i = t_hour[i]
    t_assimilated_index = np.abs(t_mod_i - t_obs).argmin()#t_obs[np.abs((t_hour[i] - t_obs)) < t_a]
    t_assimilated = t_obs[t_assimilated_index]
    delta_t = t_mod_i - t_assimilated
    C_star_updated[i] = NewtonianNudging(C_star_min, C_star_obs_lin_reg.loc[t_assimilated,:].values[0],0.5,1,delta_t,np.timedelta64(24,'h'))

In [ ]:
t_assimilated 
C_star_obs_lin_reg.loc[t_assimilated,:].values[0]

In [ ]:
type(C_star_obs_lin_reg.index.values)